## Обработка признаков, перед подачей в модель

In [7]:
import pandas as pd
import numpy as np


df_test = pd.read_parquet('/home/jupyter/datasphere/vk/test.parquet')



# Преобразование строковых дат в datetime
df_test['dates'] = df_test['dates'].apply(lambda x: pd.to_datetime(x))

# Признаки
df_test['start_date'] = df_test['dates'].apply(lambda x: x[0])  # первая дата
df_test['end_date'] = df_test['dates'].apply(lambda x: x[-1])   # последняя дата
df_test['duration_days'] = df_test['end_date'] - df_test['start_date']  # продолжительность периода

# Пример извлечения года и месяца из первой даты
df_test['start_year'] = df_test['start_date'].apply(lambda x: x.year)
df_test['start_month'] = df_test['start_date'].apply(lambda x: x.month)

# Пример извлечения среднего временного интервала между датами
df_test['mean_interval_days'] = df_test['dates'].apply(lambda x: (x[-1] - x[0]).days / len(x) if len(x) > 1 else 0)

# Агрегация
df_test['median'] = df_test['values'].apply(np.median)
df_test['mean'] = df_test['values'].apply(np.mean)
df_test['std'] = df_test['values'].apply(np.std)
df_test['min'] = df_test['values'].apply(np.min)
df_test['max'] = df_test['values'].apply(np.max)
df_test['last_value']  = df_test['values'].apply(lambda x: x[-1])
df_test['sum'] = df_test['values'].apply(np.sum)
df_test['coef_of_variation'] = df_test['std'] / df_test['mean']
df_test['ema'] = df_test['values'].apply(lambda x: pd.Series(x).ewm(span=3).mean().iloc[-1])

from scipy.stats import linregress
df_test['trend_slope'] = df_test['values'].apply(lambda x: linregress(range(len(x)), x)[0])
df_test['length'] = df_test['values'].apply(len)
df_test['kurtosis'] = df_test['values'].apply(lambda x: pd.Series(x).kurtosis())

## После обработки values и dates удалим эти столбцы

In [12]:
df_test.drop(columns = ['dates','values'], inplace = True)

## Предсказание на test

In [14]:
from catboost import CatBoostClassifier

# Загрузка модели из файла
loaded_model = CatBoostClassifier()  # или CatBoostRegressor
loaded_model.load_model('catboost_model.cbm')

# Использование загруженной модели для предсказаний
y_pred = loaded_model.predict_proba(df_test)


##сохранение предсказаний в csv файл
from IPython.display import FileLink

res = pd.DataFrame(y_pred)
res = pd.concat([df_test['id'],res], axis = 1)
result = res.rename(columns = {0:'score'})
FileLink('result.csv')

/home/jupyter/datasphere/vk/result.csv